# Import

In [867]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from matplotlib import rc
%matplotlib inline
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import matplotlib.pyplot as plt
import re
import glob
import os
from pathlib import Path

# Read csv files

In [868]:
submission = pd.read_csv('../../data/sample_submission.csv')
holidays = pd.read_csv('../../data/holidays_2023_2025.csv')
train = pd.read_csv('../../data/train/train.csv')
# os.getcwd()
terms = pd.read_csv('../solar_term_2023_2025.csv')

# Feature about date
1. year
2. month
3. day
4. weekday
5. is_holiday
6. is_sandwich
7. is_before_holiday
8. is_after_holiday
9. is_weekend

In [869]:
# 날짜 범위
dates = pd.date_range(start="2023-01-01", end="2025-05-31", freq="D")

# 연, 월, 일 col 생성
train['영업일자'] = pd.to_datetime(train['영업일자'])
train['year'] = train['영업일자'].dt.year
train['month'] = train['영업일자'].dt.month
train['day'] = train['영업일자'].dt.day
# 영업장, 메뉴명 col 생성
train[['영업장명', '메뉴명']] = train['영업장명_메뉴명'].str.split('_', expand=True)
# 요일 col 생성
train['weekday'] = train['영업일자'].dt.weekday.astype(int)

# 공휴일 col 생성
holidays["locdate"] = pd.to_datetime(holidays["locdate"])
holiday_dates = set(holidays['locdate'])
train['is_holiday'] = train['영업일자'].isin(holiday_dates).astype(int)

train['is_weekend'] = train['weekday'].apply(lambda x: 1 if x in [5, 6] else 0)

# is_sandwich: 오늘은 평일(0)이고, 어제/내일이 모두 쉬는 날(1)인 경우 1
train["non_work"] = ((train["is_holiday"] == 1) | (train["is_weekend"] == 1)).astype(int)

train["is_sandwich"] = 0
train.loc[
    (train["non_work"] == 0) &
    (train["non_work"].shift(1) == 1) &
    (train["non_work"].shift(-1) == 1),
    "is_sandwich"
] = 1


# weekday_score

In [870]:
# 요일 컬럼 만들기
train['weekday'] = pd.to_datetime(train['영업일자']).dt.weekday  # 0=월, 6=일

# 메뉴별 요일별 평균 매출 계산
menu_dow_avg = (
    train.groupby(['영업장명_메뉴명','weekday'], as_index=False)['매출수량']
         .mean()
         .rename(columns={'매출수량':'요일평균매출'})
)

# 메뉴별 min–max 스케일링 (요일 단위)
g = menu_dow_avg.groupby('영업장명_메뉴명')['요일평균매출']
menu_dow_avg['weekday_score'] = (menu_dow_avg['요일평균매출'] - g.transform('min')) / \
                             (g.transform('max') - g.transform('min'))

# max=min → 모든 요일 평균이 동일 → 1로 통일
menu_dow_avg.loc[g.transform('max') == g.transform('min'), 'weekday_score'] = 1.0

# 원본 데이터에 merge
train = train.merge(
    menu_dow_avg[['영업장명_메뉴명','weekday','weekday_score']],
    on=['영업장명_메뉴명','weekday'],
    how='left'
)

# month_score

In [871]:

# 메뉴별 월별 평균 매출 계산
menu_month_avg = (
    train.groupby(['영업장명_메뉴명','month'], as_index=False)['매출수량']
         .mean()
         .rename(columns={'매출수량':'월별평균매출'})
)

# 메뉴별 min–max 스케일링 (월 단위)
g = menu_month_avg.groupby('영업장명_메뉴명')['월별평균매출']
menu_month_avg['month_score'] = (menu_month_avg['월별평균매출'] - g.transform('min')) / \
                             (g.transform('max') - g.transform('min'))

# max=min → 모든 월 평균이 동일 → 1로 통일
menu_month_avg.loc[g.transform('max') == g.transform('min'), 'month_score'] = 1.0

# 원본 데이터에 merge
train = train.merge(
    menu_month_avg[['영업장명_메뉴명','month','month_score']],
    on=['영업장명_메뉴명','month'],
    how='left'
)

In [872]:
# 0/1로 되어 있다고 가정: non_work = (주말 or 휴일) 1, 평일 0
train['영업일자'] = pd.to_datetime(train['영업일자'])

# 1) 날짜 단위 non_work(하루에 여러 행이면 max로 대표)
daily = (
    train.groupby('영업일자', as_index=False)
         .agg({
             'non_work': 'max',     # 하루에 하나라도 주말/휴일이면 1
             'is_holiday': 'max'    # 하루에 하나라도 휴일이면 1
         })
         .sort_values('영업일자')
)

# 2) 날짜 기준으로 before/after 플래그 계산
daily['is_before_holiday'] = (
    (daily['non_work'].eq(0)) &
    (daily['non_work'].shift(-1, fill_value=0).eq(1))
).astype(int)

daily['is_after_holiday'] = (
    (daily['non_work'].eq(0)) &
    (daily['non_work'].shift(1, fill_value=0).eq(1)) &
    (daily['is_holiday'].shift(1, fill_value=0).eq(1))
).astype(int)

# 3) 원본 train에 날짜로 merge
train = train.merge(
    daily[['영업일자', 'is_before_holiday', 'is_after_holiday']],
    on='영업일자', how='left'
)

# 필요하면 non_work 컬럼 정리
train = train.drop(columns=['non_work'], errors='ignore')

# menu_rank

In [873]:
# 예시: train 데이터에 영업장명, 메뉴명, 매출수량 컬럼이 있다고 가정
# train = pd.read_csv("train.csv")

# 1) 영업장별 메뉴별 총매출
menu_sales = (
    train.groupby(['영업장명', '메뉴명'], as_index=False)['매출수량']
         .sum()
         .rename(columns={'매출수량': '총매출수량'})
)

# 2) 영업장 안에서 순위(내림차순: 많이 팔린 메뉴가 1위)
menu_sales['rank'] = menu_sales.groupby('영업장명')['총매출수량'] \
                               .rank(method='average', ascending=False)

# 3) 영업장별 메뉴 개수
menu_sales['n'] = menu_sales.groupby('영업장명')['총매출수량'].transform('size')

# 4) 0~1 스케일: (n - rank) / (n - 1)
#    → rank=1이면 1, rank=n이면 0 (n=1인 경우 예외 처리)
menu_sales['rank01'] = (menu_sales['n'] - menu_sales['rank']) / (menu_sales['n'] - 1)
menu_sales.loc[menu_sales['n'] == 1, 'rank01'] = 1.0  # 해당 영업장에 메뉴가 1개뿐이면 1

# 5) 원본 train에 붙이기
train = train.merge(
    menu_sales[['영업장명', '메뉴명', 'rank01']], 
    on=['영업장명', '메뉴명'], how='left'
).rename(columns={'rank01': 'menu_rank'})

# menu_ category

In [874]:
menu_category = {
    '1인 수저세트': '기타',
    'BBQ55(단체)': '메인메뉴',
    '대여료 60,000원': '기타',
    '대여료 30,000원': '기타',
       '대여료 90,000원': '기타',
       '본삼겹 (단품,실내)': '메인메뉴',
       '스프라이트 (단체)': '음료',
       '신라면': '추가메뉴',
       '쌈야채세트': '추가메뉴',
       '쌈장': '추가메뉴',
       '육개장 사발면': '추가메뉴',
       '일회용 소주컵': '기타',
       '일회용 종이컵': '기타',
       '잔디그늘집 대여료 (12인석)': '기타',
       '잔디그늘집 대여료 (6인석)': '기타',
       '잔디그늘집 의자 추가': '기타',
       '참이슬 (단체)': '주류',
       '친환경 접시 14cm': '기타',
       '친환경 접시 23cm': '기타',
       '카스 병(단체)': '주류',
       '콜라 (단체)': '음료',
       '햇반': '추가메뉴',
       '허브솔트': '추가메뉴',
       '(단체) 공깃밥': '추가메뉴',
       '(단체) 생목살 김치전골 2.0': '메인메뉴',
       '(단체) 은이버섯 갈비탕': '메인메뉴',
       '(단체) 한우 우거지 국밥': '메인메뉴',
       '(단체) 황태해장국 3/27까지': '메인메뉴',
       '(정식) 된장찌개': '메인메뉴',
       '(정식) 물냉면 ': '메인메뉴',
       '(정식) 비빔냉면': '메인메뉴',
       '(후식) 된장찌개': '추가메뉴',
       '(후식) 물냉면': '추가메뉴',
       '(후식) 비빔냉면': '추가메뉴',
       '갑오징어 비빔밥': '메인메뉴',
       '갱시기': '메인메뉴',
       '공깃밥': '추가메뉴',
       '꼬막 비빔밥': '메인메뉴',
       '느린마을 막걸리': '주류',
       '담하 한우 불고기': '메인메뉴',
       '담하 한우 불고기 정식': '메인메뉴',
       '더덕 한우 지짐': '메인메뉴',
       '들깨 양지탕': '메인메뉴',
       '라면사리': '추가메뉴',
       '룸 이용료': '기타',
       '메밀면 사리': '추가메뉴',
       '명인안동소주': '주류',
       '명태회 비빔냉면': '메인메뉴',
       '문막 복분자 칵테일': '주류',
       '봉평메밀 물냉면': '메인메뉴',
       '생목살 김치찌개': '메인메뉴',
       '스프라이트': '음료',
       '은이버섯 갈비탕': '메인메뉴',
       '제로콜라': '음료',
       '참이슬': '주류',
       '처음처럼': '주류',
       '카스': '주류',
       '콜라': '음료',
       '테라': '주류',
       '하동 매실 칵테일': '주류',
       '한우 떡갈비 정식': '메인메뉴',
       '한우 미역국 정식': '메인메뉴',
       '한우 우거지 국밥': '메인메뉴',
       '한우 차돌박이 된장찌개': '메인메뉴',
       '황태해장국': '메인메뉴',
       'AUS (200g)': '메인메뉴',
       'G-Charge(3)': '기타',
       'Gls.Sileni': '주류',
       'Gls.미션 서드': '주류',
       'Open Food': '기타',
       '그릴드 비프 샐러드': '메인메뉴',
       '까르보나라': '메인메뉴',
       '모둠 해산물 플래터': '메인메뉴',
       '미션 서드 카베르네 쉬라': '메인메뉴',
       '버섯 크림 리조또': '메인메뉴',
       '빵 추가 (1인)': '추가메뉴',
       '시저 샐러드 ': '메인메뉴',
       '아메리카노': '음료',
       '알리오 에 올리오 ': '메인메뉴',
       '양갈비 (4ps)': '메인메뉴',
       '자몽리치에이드': '음료',
       '하이네켄(생)': '주류',
       '한우 (200g)': '메인메뉴',
       '해산물 토마토 리조또': '메인메뉴',
       '해산물 토마토 스튜 파스타': '메인메뉴',
       '해산물 토마토 스파게티': '메인메뉴',
       '(단체)브런치주중 36,000': '메인메뉴',
       '(오븐) 하와이안 쉬림프 피자': '메인메뉴',
       '(화덕) 불고기 페퍼로니 반반피자': '메인메뉴',
       'BBQ Platter': '메인메뉴',
       'BBQ 고기추가': '추가메뉴',
       '글라스와인 (레드)': '주류',
       '레인보우칵테일(알코올)': '주류',
       '미라시아 브런치 (패키지)': '메인메뉴',
       '버드와이저(무제한)': '주류',
       '보일링 랍스타 플래터': '메인메뉴',
       '보일링 랍스타 플래터(덜매운맛)': '메인메뉴',
       '브런치 2인 패키지 ': '메인메뉴',
       '브런치 4인 패키지 ': '메인메뉴',
       '브런치(대인) 주말': '메인메뉴',
       '브런치(대인) 주중': '메인메뉴',
       '브런치(어린이)': '메인메뉴',
       '쉬림프 투움바 파스타': '메인메뉴',
       '스텔라(무제한)': '주류',
       '애플망고 에이드': '음료',
       '얼그레이 하이볼': '주류',
       '오븐구이 윙과 킬바사소세지': '메인메뉴',
       '유자 하이볼': '주류',
       '잭 애플 토닉': '주류',
       '칠리 치즈 프라이': '추가메뉴',
       '코카콜라': '음료',
       '코카콜라(제로)': '음료',
       '콥 샐러드': '추가메뉴',
       '파스타면 추가(150g)': '추가메뉴',
       '핑크레몬에이드': '음료',
       'Cass Beer': '주류',
       'Conference L1': '연회장 대여',
       'Conference L2': '연회장 대여',
       'Conference L3': '연회장 대여',
       'Conference M1': '연회장 대여',
       'Conference M8': '연회장 대여',
       'Conference M9': '연회장 대여',
       'Convention Hall': '연회장 대여',
       'Cookie Platter': '디저트',
       'Grand Ballroom': '연회장 대여',
       'OPUS 2': '연회장 대여',
       'Regular Coffee': '음료',
       '골뱅이무침': '메인메뉴',
       '돈목살 김치찌개 (밥포함)': '메인메뉴',
       '로제 치즈떡볶이': '메인메뉴',
       '마라샹궈': '메인메뉴',
       '매콤 무뼈닭발&계란찜': '메인메뉴',
       '모둠 돈육구이(3인)': '메인메뉴',
       '삼겹살추가 (200g)': '추가메뉴',
       '야채추가': '추가메뉴',
       '왕갈비치킨': '메인메뉴',
       '주먹밥 (2ea)': '추가메뉴',
       '공깃밥(추가)': '추가메뉴',
       '구슬아이스크림': '디저트',
       '단체식 13000(신)': '메인메뉴',
       '단체식 18000(신)': '메인메뉴',
       '돼지고기 김치찌개': '메인메뉴',
       '복숭아 아이스티': '음료',
       '새우 볶음밥': '메인메뉴',
       '새우튀김 우동': '메인메뉴',
       '샷 추가': '추가메뉴',
       '수제 등심 돈까스': '메인메뉴',
       '아메리카노(HOT)': '음료',
       '아메리카노(ICE)': '음료',
       '약 고추장 돌솥비빔밥': '메인메뉴',
       '어린이 돈까스': '메인메뉴',
       '오픈푸드': '기타',
       '진사골 설렁탕': '메인메뉴',
       '짜장면': '메인메뉴',
       '짜장밥': '메인메뉴',
       '짬뽕': '메인메뉴',
       '짬뽕밥': '메인메뉴',
       '치즈돈까스': '메인메뉴',
       '카페라떼(HOT)': '음료',
       '카페라떼(ICE)': '음료',
       '한상 삼겹구이 정식(2인) 소요시간 약 15~20분': '메인메뉴',
       '꼬치어묵': '메인메뉴',
       '떡볶이': '메인메뉴',
       '생수': '음료',
       '치즈 핫도그': '디저트',
       '페스츄리 소시지': '디저트',
       '단호박 식혜 ': '음료',
       '병천순대': '메인메뉴',
       '참살이 막걸리': '주류',
       '찹쌀식혜': '음료',
       '해물파전': '메인메뉴',
       '메밀미숫가루': '음료',
       '아메리카노 HOT': '음료',
       '아메리카노 ICE': '음료',
       '카페라떼 ICE': '음료',
       '현미뻥스크림': '디저트'

}


In [875]:
# Assign the menu_category to df
train['menu_category'] = train['메뉴명'].map(menu_category)
train['menu_category'], uniques = pd.factorize(train['menu_category'])

# avg_sales_all_days

In [876]:
menu_avg = train.groupby('메뉴명')['매출수량'].mean()
train['avg_sales_all_days'] = train['메뉴명'].map(menu_avg)

# solar_term, season, quarter
1. solar_term, solar_term_sum
2. season, season_sum
3. quarter, quarter_sum

In [877]:
def attach_solar_term(
    train: pd.DataFrame,
    terms: pd.DataFrame,
    date_col: str = "영업일자",
    term_date_col: str = "locdate",
    term_name_col: str = "solar_term",
    out_col: str = "solar_term"
) -> pd.DataFrame:
    df = train.copy()
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    terms = terms.copy()
    terms[term_date_col] = pd.to_datetime(terms[term_date_col], errors='coerce')

    # 절기 구간 만들기
    terms = terms.sort_values(term_date_col).reset_index(drop=True)
    terms['end_date'] = terms[term_date_col].shift(-1) - pd.Timedelta(days=1)
    terms.loc[terms.index[-1], 'end_date'] = df[date_col].max()

    # ★ 절기명 컬럼을 임시 이름으로 바꿔서 머지
    tmp_col = "_term_name_tmp"
    right = terms[[term_date_col, term_name_col, 'end_date']].rename(columns={term_name_col: tmp_col})

    merged = pd.merge_asof(
        df.sort_values(date_col),
        right.sort_values(term_date_col),
        left_on=date_col,
        right_on=term_date_col,
        direction='backward'
    )

    in_range = merged[date_col] <= merged['end_date']
    merged[out_col] = merged[tmp_col].where(in_range)

    # 임시 컬럼만 삭제 (out_col은 유지)
    merged = merged.drop(columns=[term_date_col, 'end_date', tmp_col]).sort_index()
    return merged

In [878]:
# # 0) 날짜 보정
# if df['영업일자'].dtype == 'O':
#     df['영업일자'] = pd.to_datetime(df['영업일자'], errors='coerce')

# 1) '분기' → 'quarter'로 변경 (없으면 새로 생성)
#     Q1, Q2, Q3, Q4 형식으로 생성
def get_quarter_code(m):
        if m in (1,2,3): return 0
        if m in (4,5,6): return 1
        if m in (7,8,9): return 2
        return 3
train['quarter'] = train['month'].apply(get_quarter_code)
train['season'] = train.apply(
    lambda x: 0 if x['month'] in [3, 4, 5] else
              (1 if x['month'] in [6, 7, 8] else
               (2 if x['month'] in [9, 10, 11] else 3)),
    axis=1
)

# terms 예시: columns=['locdate','절기'] 라면 term_name_col='절기'로 지정
train = attach_solar_term(
    train,
    terms,                   # 절기 lookup DF
    date_col="영업일자",
    term_date_col="locdate",
    term_name_col="solar_term",    # 실제 컬럼명 맞추기!
    out_col="solar_term"     # train에 새로 생길 이름
)


In [879]:
def cumsum(train, col, newcol):    
# 0) 날짜 보정
    if train['영업일자'].dtype == 'O':
        train['영업일자'] = pd.to_datetime(train['영업일자'], errors='coerce')

    # 2) 임시로 매장/메뉴 분리(컬럼 추가 X)
    keys = train['영업장명_메뉴명'].str.split('_', n=1, expand=True)
    g_store, g_menu = keys[0], keys[1]

    # 3) 그룹별 전일까지 누적합 → 전체 열에 한 번에 대입 (정수 고정)
    #    - 정렬은 누적 순서만 위해 잠깐 사용, 결과는 원래 인덱스로 돌아옴
    train_sorted = train.sort_values([col, '영업일자']).copy()
    seasonal_series = (
        train_sorted
        .groupby([g_store.reindex(train_sorted.index),
                    g_menu.reindex(train_sorted.index),
                    train_sorted[col]], sort=False)['매출수량']
        .transform(lambda s: s.shift(1).cumsum())
    )

    train[newcol] = (
        seasonal_series.reindex(train_sorted.index)
                    .reindex(train.index)
                    .fillna(0)
                    .astype('int64')
    )
    return train
train = cumsum(train, 'quarter', 'quarter_sum')
train = cumsum(train, 'season', 'season_sum')
train = cumsum(train, 'solar_term', 'solar_term_sum')


# Feature about zero
1. avg_sales_nonzero_days
2. avg_sales_nonzero_monthly
3. var_sales_nonzero_monthly
4. zero_sales_day_ratio
5. avg_sales_nonzero_seasonly
6. avg_sales_nonzero_weekday

In [880]:
nonzero_avg = train[train['매출수량'] > 0].groupby('메뉴명')['매출수량'].mean()
# Assign the nonzero_avg to df
train['avg_sales_nonzero_days'] = train['메뉴명'].map(nonzero_avg)
# 매출수량 > 0인 경우만 메뉴별 분산 계산
nonzero_var = (
    train[train['매출수량'] > 0]
    .groupby('메뉴명')['매출수량']
    .var()   # ← 분산
)
nonzero_avg_monthly = (
    train[train['매출수량'] > 0]
    .groupby(['메뉴명', 'year', 'month'])['매출수량']
    .mean()
    .rename('avg_sales_nonzero_monthly')
    .reset_index()
)

train = train.merge(nonzero_avg_monthly, on=['메뉴명', 'year', 'month'], how='left')
train['avg_sales_nonzero_monthly'] = train['avg_sales_nonzero_monthly'].fillna(0)

# train에 새로운 칼럼으로 추가
train['var_sales_nonzero_days'] = train['메뉴명'].map(nonzero_var)

In [881]:
zero_ratio = train.groupby('메뉴명')['매출수량'].apply(lambda x: (x.eq(0).sum() / len(x)) * 100)

# Assign the zero_ratio to df
train['zero_sales_day_ratio'] = train['메뉴명'].map(zero_ratio)

In [882]:
nonzero_avg_seasonly = (
    train[train['매출수량'] > 0]
    .groupby(['메뉴명', 'season'])['매출수량']
    .mean()
    .rename('avg_sales_nonzero_seasonly')
)
train = train.merge(nonzero_avg_seasonly, on=['메뉴명', 'season'], how='left')
nonzero_avg_weekday = (
    train[train['매출수량'] > 0]
    .groupby(['메뉴명', 'weekday'])['매출수량']
    .mean()
    .rename('avg_sales_nonzero_weekday')
)
train = train.merge(nonzero_avg_weekday, on=['메뉴명', 'weekday'], how='left')
train['avg_sales_nonzero_weekday'] = train['avg_sales_nonzero_weekday'].fillna(0)
train['avg_sales_nonzero_seasonly'] = train['avg_sales_nonzero_weekday'].fillna(0)

train = train.sort_values(['영업장명_메뉴명', '영업일자']).reset_index(drop=True)

# demand_volatility

In [883]:
menu_stats=(
    train.groupby('영업장명_메뉴명')['매출수량']
    .agg(['mean','std'])
    .reset_index()
)

menu_stats['demand_volatility']=menu_stats['std']/menu_stats['mean']

menu_stats.rename(columns={'mean':'평균매출수량','std':'표준편차'},inplace=True)

# Merge menu_stats back to df
train=train.merge(menu_stats[['영업장명_메뉴명', 'demand_volatility']], on='영업장명_메뉴명',how='left')

# date_weight

In [884]:
train = train.sort_values(['영업장명_메뉴명', '영업일자']).reset_index(drop=True)
df = train.copy().sort_values(['영업장명_메뉴명','영업일자']).reset_index(drop=True)
# 필요 컬럼 가정: ['영업장명','영업일자','is_holiday','is_weekend','weekday','month','day']
# weekday: 월=0, 화=1, ... 일=6

# ─────────────────────────────────────────────────────
# 보조 플래그: 영업장별로 sandwich / adjacent 계산
# ─────────────────────────────────────────────────────
df['is_special'] = ((df['is_holiday']==1) | (df['is_weekend']==1)).astype(int)

def mark_sandwich_adjacent(g: pd.DataFrame) -> pd.DataFrame:
    prev_special = g['is_special'].shift(1, fill_value=0)
    next_special = g['is_special'].shift(-1, fill_value=0)
    g['is_sandwich'] = ((prev_special.eq(1)) & (g['is_special'].eq(0)) & (next_special.eq(1))).astype(int)
    g['is_adjacent'] = ((g['is_special'].eq(0)) & (prev_special.eq(1) | next_special.eq(1))).astype(int)
    return g

df = df.groupby('영업장명', group_keys=False).apply(mark_sandwich_adjacent)

# ─────────────────────────────────────────────────────
# 규칙 → 마스크 빌더
# ─────────────────────────────────────────────────────
def build_mask(name: str, g: pd.DataFrame, rule: dict):
    """name 키워드와 rule 파라미터로 boolean mask 생성"""
    if name == 'holiday':
        return g['is_holiday'].eq(1)
    if name == 'weekend':
        return (g['is_holiday'].eq(0) & g['is_weekend'].eq(1))
    if name == 'sandwich':
        return (g['is_holiday'].eq(0) & g['is_weekend'].eq(0) & g['is_sandwich'].eq(1))
    if name == 'adjacent':
        return (g['is_holiday'].eq(0) & g['is_weekend'].eq(0) & g['is_adjacent'].eq(1))
    if name == 'friday':
        return (g['is_holiday'].eq(0) & g['is_weekend'].eq(0) & g['weekday'].eq(4))
    if name == 'tuewedthu':
        return (g['is_holiday'].eq(0) & g['is_weekend'].eq(0) & g['weekday'].isin([1,2,3]))
    if name == 'monday':
        return (g['is_holiday'].eq(0) & g['is_weekend'].eq(0) & g['weekday'].eq(0))
    if name == 'weekday_in':
        days = rule.get('days', [])
        return (g['is_holiday'].eq(0) & g['is_weekend'].eq(0) & g['weekday'].isin(days))
    if name == 'custom_mask':
        return rule['func'](g)

    # ---- 여기부터 추가된 복합 규칙들 ----
    # 화~금 (평일만, 휴일/주말 제외)
    if name == 'tuewedthufri':
        return (g['is_holiday'].eq(0) & g['is_weekend'].eq(0) & g['weekday'].isin([1,2,3,4]))
    # 월~목 (평일만, 휴일/주말 제외)
    if name == 'montuewedthu':
        return (g['is_holiday'].eq(0) & g['is_weekend'].eq(0) & g['weekday'].isin([0,1,2,3]))
    # 목/금 (휴일 여부 무관하게 요일만 기준; 위에서 공휴일이 먼저 매칭되므로 충돌 없음)
    if name == 'thufri':
        return g['weekday'].isin([3,4])
    # 화/토 (토요일은 주말이지만, 휴일은 제외하여 공휴일 규칙 우선)
    if name == 'tuesat':
        return (g['is_holiday'].eq(0) & g['weekday'].isin([1,5]))
    # 월/화/수/토/일 (휴일 제외; 연회장/BBQ용)
    if name == 'montuewedsatsun':
        return (g['is_holiday'].eq(0) & g['weekday'].isin([0,1,2,5,6]))
    # 월/일 (휴일 제외)
    if name == 'monsun':
        return (g['is_holiday'].eq(0) & g['weekday'].isin([0,6]))

    raise ValueError(f"Unknown rule name: {name}")

def apply_date_rules(g: pd.DataFrame, rules: list, default_val: float=1.0) -> pd.Series:
    """우선순위대로 date_mult를 채움"""
    out = pd.Series(default_val, index=g.index, dtype=float)
    assigned = pd.Series(False, index=g.index)
    for r in rules:
        name = r['name']
        mult = r['mult']
        mask = build_mask(name, g, r) & (~assigned)
        out.loc[mask] = mult
        assigned.loc[mask] = True
    return out

def apply_month_rules(g: pd.DataFrame, rules: list, default_val: float=1.0) -> pd.Series:
    """여러 월/일 조건을 병렬로 적용(후속 규칙이 덮어씀)"""
    out = pd.Series(default_val, index=g.index, dtype=float)
    for r in rules:
        mult = r['mult']
        if 'months' in r:
            mask = g['month'].isin(r['months'])
        elif 'month' in r and ('day_from' in r or 'day_to' in r):
            # 단일 월 + 일자 범위
            mask = g['month'].eq(r['month'])
            if 'day_from' in r:
                mask &= g['day'] >= r['day_from']
            if 'day_to' in r:
                mask &= g['day'] <= r['day_to']
        elif 'custom_mask' in r:
            mask = r['custom_mask'](g)
        else:
            continue
        out.loc[mask] = mult
    return out

# ─────────────────────────────────────────────────────
# 매장별 CONFIG 예시
#   - 이 블록만 채우면 전체 자동
# ─────────────────────────────────────────────────────
CONFIG = {
    # 예시) 포레스트릿
    '포레스트릿': {
        'date': {
            'default': 1.0,
            'rules': [
                {'name':'holiday',   'mult':1.35},
                {'name':'weekend',   'mult':1.20},
                {'name':'sandwich',  'mult':1.10},
                {'name':'adjacent',  'mult':1.06},
                {'name':'friday',    'mult':1.03},
                {'name':'tuewedthu', 'mult':1.00},
            ]
        },
        'month': {
            'default': 1.0,
            'rules': [
                {'months':[1,2,12],        'mult':1.20},
                {'months':[4,5,6,9,10,11], 'mult':1.10},
                {'months':[3,7,8],         'mult':0.92},
            ]
        }
    },
    '카페테리아': {
        'date': {
            'default': 1.0,
            'rules': [
                {'name':'holiday',   'mult':1.35},
                {'name':'weekend',   'mult':1.20},
                {'name':'sandwich',  'mult':1.10},
                {'name':'adjacent',  'mult':1.06},
                {'name':'friday',    'mult':1.03},
                {'name':'tuewedthu', 'mult':1.00},
            ]
        },
        'month': {
            'default': 1.0,
            'rules': [
                {'months':[1,2,12],         'mult':1.20},
                {'months':[9,10],           'mult':1.10},
                {'months':[3,4,5,6,7,8,11], 'mult':0.92},
            ]
        }
    },
    # (예시) 미라시아 — 규칙이 다르다고 가정
    '화담숲주막': {
        'date': {
            'default': 1.0,
            'rules': [
                {'name':'holiday',   'mult':1.35},   # 공휴일 더 강하게
                {'name':'weekend',   'mult':1.20},
                {'name':'sandwich',  'mult':1.10},
                {'name':'adjacent',  'mult':1.06},
                {'name':'tuewedthufri', 'mult':1.00},
                {'name':'monday',    'mult':0.97},   # 월요일 약세 반영
            ]
        },
        'month': {
            'default': 1.0,
            'rules': [
                {'months':[4,5,6,10,11],         'mult':1.20},
                {'months':[7,8,9],           'mult':1.10},
                {'months':[1,2,3,12], 'mult':0.92},
            ]
        }
    },
    '화담숲카페': {
        'date': {
            'default': 1.0,
            'rules': [
                {'name':'holiday',   'mult':1.35},   # 공휴일 더 강하게
                {'name':'weekend',   'mult':1.20},
                {'name':'sandwich',  'mult':1.10},
                {'name':'adjacent',  'mult':1.06},
                {'name':'tuewedthufri', 'mult':1.00},
                {'name':'monday',    'mult':0.97},   # 월요일 약세 반영
            ]
        },
        'month': {
            'default': 1.0,
            'rules': [
                {'months':[4,5,6,10,11],         'mult':1.20},
                {'months':[7,8,9],           'mult':1.10},
                {'months':[1,2,3,12], 'mult':0.92},
            ]
        }
    },
    '미라시아': {
        'date': {
            'default': 1.0,
            'rules': [
                {'name':'holiday',   'mult':1.35},   # 공휴일 더 강하게
                {'name':'weekend',   'mult':1.20},
                {'name':'sandwich',  'mult':1.10},
                {'name':'adjacent',  'mult':1.06},
                {'name':'friday', 'mult':1.03},
                {'name':'montuewedthu',    'mult':1.00},   # 월요일 약세 반영
            ]
        },
        'month': {
            'default': 1.0,
            'rules': [
                {'months':[1,2,9,10,11],         'mult':1.20},
                {'months':[8],           'mult':1.10},
                {'months':[4,5,6,7,12], 'mult':1.05},
                {'months':[3], 'mult':0.92},
            ]
        }
    },
    '담하': {
        'date': {
            'default': 1.0,
            'rules': [
                {'name':'holiday',   'mult':1.35},   # 공휴일 더 강하게
                {'name':'weekend',   'mult':1.20},
                {'name':'sandwich',  'mult':1.10},
                {'name':'adjacent',  'mult':1.06},
                {'name':'friday', 'mult':1.03},
                {'name':'montuewedthu',    'mult':1.00},   # 월요일 약세 반영
            ]
        },
        'month': {
            'default': 1.0,
            'rules': [
                {'months':[1,2,10,11],         'mult':1.20},
                {'months':[4,5,6,7,8,9,12], 'mult':1.10},
                {'months':[3], 'mult':0.92},
            ]
        }
    },
    '라그로타': {
        'date': {
            'default': 1.0,
            'rules': [
                {'name':'holiday',   'mult':1.35},   # 공휴일 더 강하게
                {'name':'weekend',   'mult':1.20},
                {'name':'sandwich',  'mult':1.10},
                {'name':'adjacent',  'mult':1.06},
                {'name':'friday', 'mult':1.03},
                {'name':'tuewedthu',    'mult':1.00},   # 월요일 약세 반영
                {'name':'monday',    'mult':0.97},   # 월요일 약세 반영
            ]
        },
        'month': {
            'default': 1.0,
            'rules': [
                {'months':[1,2,12],         'mult':1.20},
                {'months':[4,5,6,7,8,9,10,11], 'mult':1.10},
                {'months':[3], 'mult':0.92},
            ]
        }
    },
    '느티나무 셀프BBQ': {
        'date': {
            'default': 1.0,
            'rules': [
                {'name':'thufri',   'mult':1.20},   # 공휴일 더 강하게
                {'name':'montuewedsatsun',   'mult':1.00},
            ]
        },
        'month': {
            'default': 1.0,
            'rules': [
                {'months':[4,5,6,7,9,10,11,12],         'mult':1.20},
                {'months':[1,2,3,8], 'mult':1.00},
            ]
        }
    },
    '연회장': {
        'date': {
            'default': 1.0,
            'rules': [
                {'name':'thufri',   'mult':1.20},   # 공휴일 더 강하게
                {'name':'tuesat',   'mult':1.10},
                {'name':'monsun',   'mult':1.00},
            ]
        },
        'month': {
            'default': 1.0,
            'rules': [
                {'months':[1,2,3,4,5,6,7,8,9,10,11,12], 'mult':1.00},
            ]
        }
    },
    
}

# ─────────────────────────────────────────────────────
# 적용
# ─────────────────────────────────────────────────────
df['date_mult']  = 1.0
df['month_mult'] = 1.0



for store, g in df.groupby('영업장명'):
    if store not in CONFIG:
        raise KeyError(f"CONFIG에 '{store}'가 없습니다.")
    conf = CONFIG[store]
    date_mult  = apply_date_rules(g, conf['date']['rules'],  conf['date'].get('default', 1.0))
    month_mult = apply_month_rules(g, conf['month']['rules'], conf['month'].get('default', 1.0))
    df.loc[g.index, 'date_mult']  = date_mult.values
    df.loc[g.index, 'month_mult'] = month_mult.values
    

# 최종 배수 및 안정화
df['date_weight'] = (df['date_mult'] * df['month_mult']).clip(0.7, 1.6)

train = train.sort_values(['영업장명_메뉴명', '영업일자']).reset_index(drop=True)

# 결과
train['date_weight'] = df['date_weight'].values

/var/folders/3t/x_vcdczj5fn_v1nnkppzjpwh0000gn/T/ipykernel_10003/3133154072.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('영업장명', group_keys=False).apply(mark_sandwich_adjacent)


# menu_date_weight

In [885]:
def build_weight_map_from_train(df_train: pd.DataFrame) -> pd.DataFrame:
    """
    입력 df_train에는 '영업일자', '영업장명_메뉴명', '매출수량' 컬럼이 있어야 한다.
    산출물은 (영업장명_메뉴명, weekday, weekday_weight) 매핑 테이블.
    """
    df = df_train.copy()

    # 메뉴×요일 평균
    by_wk = (df.groupby(['영업장명_메뉴명','weekday'], as_index=False)['매출수량']
               .mean()
               .rename(columns={'매출수량':'wk_mean'}))

    # 메뉴 전체 평균
    by_menu = (df.groupby('영업장명_메뉴명', as_index=False)['매출수량']
                 .mean()
                 .rename(columns={'매출수량':'overall_mean'}))

    merged = by_wk.merge(by_menu, on='영업장명_메뉴명', how='left')
    merged['dev'] = merged['wk_mean'] - merged['overall_mean']  # 방향성 있는 편차

    # 메뉴 내부에서 1~2 스케일
    def scale_1_2(g):
        mn = g['dev'].min()
        mx = g['dev'].max()
        if np.isclose(mx, mn):
            g['weekday_weight'] = 1.0  # 전부 동일하면 1.0
        else:
            g['weekday_weight'] = 1 + (g['dev'] - mn) / (mx - mn)
        return g

    weighted = merged.groupby('영업장명_메뉴명', group_keys=False).apply(scale_1_2)
    weight_map = weighted[['영업장명_메뉴명','weekday','weekday_weight']].copy()

    # 안전 검사 (0 금지, NaN 금지)
    if (weight_map['weekday_weight'] <= 0).any():
        raise ValueError("weekday_weight에 0 이하 값이 포함되었습니다. 로직을 점검하세요.")
    if weight_map['weekday_weight'].isna().any():
        raise ValueError("weekday_weight에 NaN이 포함되었습니다.")

    return weight_map

weight_map = build_weight_map_from_train(train)
train = train.merge(weight_map, on=['영업장명_메뉴명','weekday'], how='left')

def build_month_weight_map_from_train(df_train: pd.DataFrame) -> pd.DataFrame:
    """
    입력 df_train에는 '영업일자', '영업장명_메뉴명', '매출수량', 'month' 컬럼이 있어야 한다.
    산출물은 (영업장명_메뉴명, month, month_weight) 매핑 테이블.
    """
    df = df_train.copy()

    # 메뉴×월 평균
    by_m = (df.groupby(['영업장명_메뉴명','month'], as_index=False)['매출수량']
              .mean()
              .rename(columns={'매출수량':'m_mean'}))

    # 메뉴 전체 평균
    by_menu = (df.groupby('영업장명_메뉴명', as_index=False)['매출수량']
                 .mean()
                 .rename(columns={'매출수량':'overall_mean'}))

    merged = by_m.merge(by_menu, on='영업장명_메뉴명', how='left')
    merged['dev'] = merged['m_mean'] - merged['overall_mean']

    def scale_1_2(g):
        mn, mx = g['dev'].min(), g['dev'].max()
        if np.isclose(mx, mn):
            g['month_weight'] = 1.0
        else:
            g['month_weight'] = 1 + (g['dev'] - mn) / (mx - mn)
        return g

    weighted = merged.groupby('영업장명_메뉴명', group_keys=False).apply(scale_1_2)
    weight_map = weighted[['영업장명_메뉴명','month','month_weight']].copy()

    if (weight_map['month_weight'] <= 0).any():
        raise ValueError("month_weight에 0 이하 값이 포함되었습니다.")
    if weight_map['month_weight'].isna().any():
        raise ValueError("month_weight에 NaN이 포함되었습니다.")

    return weight_map
month_weight_map = build_month_weight_map_from_train(train)
train = train.merge(month_weight_map, on=['영업장명_메뉴명','month'], how='left')

/var/folders/3t/x_vcdczj5fn_v1nnkppzjpwh0000gn/T/ipykernel_10003/2826190629.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weighted = merged.groupby('영업장명_메뉴명', group_keys=False).apply(scale_1_2)
/var/folders/3t/x_vcdczj5fn_v1nnkppzjpwh0000gn/T/ipykernel_10003/2826190629.py:73: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weighted = merged.groupby('영업장명_메뉴명', group_keys=False).apply(scale_1_2)


In [886]:
# weekday_weight × month_weight → menu_date_weight
train['menu_date_weight'] = train['weekday_weight'] * train['month_weight']
train = train.drop(columns=['weekday_weight','month_weight'])

# Store trian.csv

In [887]:
train[['영업장명', '메뉴명']] = train['영업장명_메뉴명'].str.split('_', expand=True)

train = train.sort_values(['영업장명_메뉴명', '영업일자']).reset_index(drop=True)
train.to_csv("train_fin.csv", index=False, encoding="utf-8-sig")

print(train.columns)

print("train DataFrame이 output.csv로 저장되었습니다.")

Index(['영업일자', '영업장명_메뉴명', '매출수량', 'year', 'month', 'day', '영업장명', '메뉴명',
       'weekday', 'is_holiday', 'is_weekend', 'is_sandwich', 'weekday_score',
       'month_score', 'is_before_holiday', 'is_after_holiday', 'menu_rank',
       'menu_category', 'avg_sales_all_days', 'quarter', 'season',
       'solar_term', 'quarter_sum', 'season_sum', 'solar_term_sum',
       'avg_sales_nonzero_days', 'avg_sales_nonzero_monthly',
       'var_sales_nonzero_days', 'zero_sales_day_ratio',
       'avg_sales_nonzero_seasonly', 'avg_sales_nonzero_weekday',
       'demand_volatility', 'date_weight', 'menu_date_weight'],
      dtype='object')
train DataFrame이 output.csv로 저장되었습니다.


# Task
Apply the same feature engineering steps (date features, holiday features, spike/drop, seasonal index, banquet type, etc.) that were applied to the training data to the following test files: "TEST_01.csv", "TEST_02.csv", "TEST_03.csv", "TEST_04.csv", "TEST_05.csv", "TEST_06.csv", "TEST_07.csv", "TEST_08.csv", "TEST_09.csv". Ensure that all engineered features are added as new columns to the respective DataFrames loaded from these files.

## Identify test files

### Subtask:
Create a list of all the test file paths (`TEST_01.csv` to `TEST_09.csv`).


**Reasoning**:
Create a list containing the file paths for the test datasets.



In [888]:
test_files = [f"TEST_{i:02d}.csv" for i in range(0, 10)] # Changed range from 1 to 0 to include TEST_00
print(test_files)

['TEST_00.csv', 'TEST_01.csv', 'TEST_02.csv', 'TEST_03.csv', 'TEST_04.csv', 'TEST_05.csv', 'TEST_06.csv', 'TEST_07.csv', 'TEST_08.csv', 'TEST_09.csv']


## Define feature engineering function

### Subtask:
Create a function that takes a DataFrame (like the one loaded from a test file) and applies all the necessary feature engineering steps (date features, holiday features, spike/drop, seasonal index, banquet type, etc.) to it, returning the processed DataFrame.


**Reasoning**:
Define a function `engineer_features` that takes a DataFrame and applies all the feature engineering steps. This function will include date features, holiday features, spike/drop detection, weekday/weekend price flags, seasonal index, brunch/hallroom flags, and banquet type merging.



In [ ]:
def engineer_features(df_test, holiday_df, terms_df=None, terms_csv_path=None):
    """Applies feature engineering steps to a test DataFrame.
       terms_df: (선택) 절기 테이블 DataFrame (cols: ['locdate','solar_term'])
       terms_csv_path: (선택) terms_df가 없을 때 읽어올 CSV 경로
    """
    import pandas as pd
    df_test[['영업장명', '메뉴명']] = df_test['영업장명_메뉴명'].str.split('_', expand=True)


    # --- 기본 날짜 피처 ---
    # year, month, day, weekday
    df_test = df_test.copy()
    df_test['영업일자'] = pd.to_datetime(df_test['영업일자'], errors='coerce')
    df_test['year']    = df_test['영업일자'].dt.year.astype(int)
    df_test['month']   = df_test['영업일자'].dt.month.astype(int)
    df_test['day']     = df_test['영업일자'].dt.day.astype(int)
    df_test['weekday'] = df_test['영업일자'].dt.weekday.astype(int)

    # season, quarter, solar_term
    def get_season_code(m):
        if m in (12,1,2): return 0
        if m in (3,4,5):  return 1
        if m in (6,7,8):  return 2
        return 3
    df_test['season'] = df_test['month'].apply(get_season_code)
    def get_quarter_code(m):
        if m in (1,2,3): return 0
        if m in (4,5,6): return 1
        if m in (7,8,9): return 2
        return 3
    df_test['quarter'] = df_test['month'].apply(get_quarter_code)
    df_test = attach_solar_term(
        df_test,
        terms,                   # 절기 lookup DF
        date_col="영업일자",
        term_date_col="locdate",
        term_name_col="solar_term",    # 실제 컬럼명 맞추기!
        out_col="solar_term"     # train에 새로 생길 이름
    )

    # season_sum, quarter_sum, solar_term_sum
    def attach_avg_by_keys(train, df_test, value_col, out_col=None,
                        keys=('영업장명_메뉴명','month','day')):
        # train에서 키별 평균 테이블
        mp = (train.groupby(list(keys), as_index=False)[value_col]
                    .mean()
                    .rename(columns={value_col: out_col or value_col}))
        # test에 merge
        df_test = df_test.merge(mp, on=list(keys), how='left')
        return df_test

    # 사용 예시: 각 *_sum 컬럼의 평균을 동일 키로 test에 붙이기
    df_test = attach_avg_by_keys(train, df_test, 'season_sum')
    df_test = attach_avg_by_keys(train, df_test, 'solar_term_sum')
    df_test = attach_avg_by_keys(train, df_test, 'quarter_sum')

    # is_holiday, is_weekend, is_sandwich, is_before_holiday, is_after_holiday
    holiday_df['locdate'] = pd.to_datetime(holiday_df['locdate'])
    df_test = pd.merge(
        df_test,
        holiday_df[['locdate', 'isHoliday']],
        how='left',
        left_on='영업일자',
        right_on='locdate'
    )
    df_test['is_holiday'] = df_test['isHoliday'].fillna('N').apply(lambda x: 1 if x == 'Y' else 0)
    df_test = df_test.drop(['locdate', 'isHoliday'], axis=1)  # 둘 다 삭제
    df_test['is_weekend'] = df_test['weekday'].apply(lambda x: 1 if x in [5, 6] else 0)

    df_test["non_work"] = ((df_test["is_holiday"] == 1) | (df_test["is_weekend"] == 1)).astype(int)

    # menu_rank
    menu_rank_map = train[['영업장명_메뉴명', 'menu_rank']].drop_duplicates()
    df_test = df_test.merge(
        menu_rank_map,
        on='영업장명_메뉴명',
        how='left'
    )

    # 0/1로 정의: non_work = (주말 or 휴일) 이면 1, 평일이면 0
    df_test['영업일자'] = pd.to_datetime(df_test['영업일자'])
    daily = (df_test.groupby('영업일자', as_index=False)['non_work']
                .max()
                .sort_values('영업일자'))

    # is_sandwich
    daily['is_sandwich'] = (
        (daily['non_work'].eq(0)) &
        (daily['non_work'].shift(1,  fill_value=0).eq(1)) &
        (daily['non_work'].shift(-1, fill_value=0).eq(1))
    ).astype(int)

    # 원본 df_test에 날짜로 merge (기존 컬럼 있으면 덮어쓰기)
    df_test = df_test.drop(columns=['is_sandwich'], errors='ignore') \
                    .merge(daily[['영업일자','is_sandwich']], on='영업일자', how='left')
    # 0/1로 되어 있다고 가정: non_work = (주말 or 휴일) 1, 평일 0
    df_test['영업일자'] = pd.to_datetime(df_test['영업일자'])

    # 날짜 단위 non_work(하루에 여러 행이면 max로 대표)
    daily = (
        df_test.groupby('영업일자', as_index=False)
            .agg({
                'non_work': 'max',     # 하루에 하나라도 주말/휴일이면 1
                'is_holiday': 'max'    # 하루에 하나라도 휴일이면 1
            })
            .sort_values('영업일자')
    )

    # 날짜 기준으로 before/after 플래그 계산
    daily['is_before_holiday'] = (
        (daily['non_work'].eq(0)) &
        (daily['non_work'].shift(-1, fill_value=0).eq(1))
    ).astype(int)

    daily['is_after_holiday'] = (
        (daily['non_work'].eq(0)) &
        (daily['non_work'].shift(1, fill_value=0).eq(1)) &
        (daily['is_holiday'].shift(1, fill_value=0).eq(1))
    ).astype(int)
    df_test = df_test.merge(
        daily[['영업일자', 'is_before_holiday', 'is_after_holiday']],
        on='영업일자', how='left'
    )
    df_test = df_test.drop(columns=['non_work'], errors='ignore')

    # 타입/정렬
    df_test['영업일자'] = pd.to_datetime(df_test['영업일자'])
    df_test = df_test.sort_values(['영업장명','영업일자'])

    # lookup features
    # menu_category, avg_sales_all_days, avg_sales_nonzero_days, zero_sales_day_ratio, var_sales_nonzero_days, demand_volatility

    feat_cols = ['영업장명_메뉴명',
             'menu_category','avg_sales_all_days','avg_sales_nonzero_days',
             'zero_sales_day_ratio', 'var_sales_nonzero_days', "demand_volatility"]
    train_feats = (train[feat_cols]
               .groupby('영업장명_메뉴명', as_index=False)
               .agg('first'))  # 필요시 mean/max로 변경
    df_test = df_test.merge(train_feats, on='영업장명_메뉴명',
                        how='left', validate='many_to_one')

    # menu_date_weight
    trainfeats = (
        train.groupby(['영업장명_메뉴명', 'month', 'weekday'])['menu_date_weight']
            .first()              # 값이 항상 동일하다면 first OK (혹시 다르면 mean 권장)
            .reset_index()
    )
    df_test = df_test.merge(
        trainfeats,
        on=['영업장명_메뉴명', 'month', 'weekday'],
        how='left',
        validate='many_to_one'       # test 다대, trainfeats 일대
    )

    # avg_sales_nonzero_monthly
    trainfeats = (
        train.groupby(['영업장명_메뉴명', 'month'])['avg_sales_nonzero_monthly']
            .first()              # 값이 항상 동일하다면 first OK (혹시 다르면 mean 권장)
            .reset_index()
    )
    df_test = df_test.merge(
        trainfeats,
        on=['영업장명_메뉴명', 'month'],
        how='left',
        validate='many_to_one'       # test 다대, trainfeats 일대
    )
    
    # avg_sales_nonzero_seasonly
    trainfeats = (
        train.groupby(['영업장명_메뉴명', 'season'])['avg_sales_nonzero_seasonly']
            .first()              # 값이 항상 동일하다면 first OK (혹시 다르면 mean 권장)
            .reset_index()
    )
    df_test = df_test.merge(
        trainfeats,
        on=['영업장명_메뉴명', 'season'],
        how='left',
        validate='many_to_one'       # test 다대, trainfeats 일대
    )

    # avg_sales_nonzero_weekday
    trainfeats = (
        train.groupby(['영업장명_메뉴명', 'weekday'])['avg_sales_nonzero_weekday']
            .first()              # 값이 항상 동일하다면 first OK (혹시 다르면 mean 권장)
            .reset_index()
    )
    df_test = df_test.merge(
        trainfeats,
        on=['영업장명_메뉴명', 'weekday'],
        how='left',
        validate='many_to_one'       # test 다대, trainfeats 일대
    )

    # weekday_score
    trainfeats = (
        train.groupby(['영업장명_메뉴명', 'weekday'])['weekday_score']
            .first()              # 값이 항상 동일하다면 first OK (혹시 다르면 mean 권장)
            .reset_index()
    )
    df_test = df_test.merge(
        trainfeats,
        on=['영업장명_메뉴명', 'weekday'],
        how='left',
        validate='many_to_one'       # test 다대, trainfeats 일대
    )

    # month_score
    trainfeats = (
        train.groupby(['영업장명_메뉴명', 'month'])['month_score']
            .first()              # 값이 항상 동일하다면 first OK (혹시 다르면 mean 권장)
            .reset_index()
    )
    df_test = df_test.merge(
        trainfeats,
        on=['영업장명_메뉴명', 'month'],
        how='left',
        validate='many_to_one'       # test 다대, trainfeats 일대
    )

    
    # date_weight
    df_test = df_test.sort_values(['영업장명_메뉴명', '영업일자']).reset_index(drop=True)
    df = df_test.copy().sort_values(['영업장명_메뉴명','영업일자']).reset_index(drop=True)
    df['is_special'] = ((df['is_holiday']==1) | (df['is_weekend']==1)).astype(int)

    def mark_sandwich_adjacent(g: pd.DataFrame) -> pd.DataFrame:
        prev_special = g['is_special'].shift(1, fill_value=0)
        next_special = g['is_special'].shift(-1, fill_value=0)
        g['is_sandwich'] = ((prev_special.eq(1)) & (g['is_special'].eq(0)) & (next_special.eq(1))).astype(int)
        g['is_adjacent'] = ((g['is_special'].eq(0)) & (prev_special.eq(1) | next_special.eq(1))).astype(int)
        return g

    df = df.groupby('영업장명', group_keys=False).apply(mark_sandwich_adjacent)


    def build_mask(name: str, g: pd.DataFrame, rule: dict):
        """name 키워드와 rule 파라미터로 boolean mask 생성"""
        if name == 'holiday':
            return g['is_holiday'].eq(1)
        if name == 'weekend':
            return (g['is_holiday'].eq(0) & g['is_weekend'].eq(1))
        if name == 'sandwich':
            return (g['is_holiday'].eq(0) & g['is_weekend'].eq(0) & g['is_sandwich'].eq(1))
        if name == 'adjacent':
            return (g['is_holiday'].eq(0) & g['is_weekend'].eq(0) & g['is_adjacent'].eq(1))
        if name == 'friday':
            return (g['is_holiday'].eq(0) & g['is_weekend'].eq(0) & g['weekday'].eq(4))
        if name == 'tuewedthu':
            return (g['is_holiday'].eq(0) & g['is_weekend'].eq(0) & g['weekday'].isin([1,2,3]))
        if name == 'monday':
            return (g['is_holiday'].eq(0) & g['is_weekend'].eq(0) & g['weekday'].eq(0))
        if name == 'weekday_in':
            days = rule.get('days', [])
            return (g['is_holiday'].eq(0) & g['is_weekend'].eq(0) & g['weekday'].isin(days))
        if name == 'custom_mask':
            return rule['func'](g)

        # ---- 여기부터 추가된 복합 규칙들 ----
        # 화~금 (평일만, 휴일/주말 제외)
        if name == 'tuewedthufri':
            return (g['is_holiday'].eq(0) & g['is_weekend'].eq(0) & g['weekday'].isin([1,2,3,4]))
        # 월~목 (평일만, 휴일/주말 제외)
        if name == 'montuewedthu':
            return (g['is_holiday'].eq(0) & g['is_weekend'].eq(0) & g['weekday'].isin([0,1,2,3]))
        # 목/금 (휴일 여부 무관하게 요일만 기준; 위에서 공휴일이 먼저 매칭되므로 충돌 없음)
        if name == 'thufri':
            return g['weekday'].isin([3,4])
        # 화/토 (토요일은 주말이지만, 휴일은 제외하여 공휴일 규칙 우선)
        if name == 'tuesat':
            return (g['is_holiday'].eq(0) & g['weekday'].isin([1,5]))
        # 월/화/수/토/일 (휴일 제외; 연회장/BBQ용)
        if name == 'montuewedsatsun':
            return (g['is_holiday'].eq(0) & g['weekday'].isin([0,1,2,5,6]))
        # 월/일 (휴일 제외)
        if name == 'monsun':
            return (g['is_holiday'].eq(0) & g['weekday'].isin([0,6]))

        raise ValueError(f"Unknown rule name: {name}")

    def apply_date_rules(g: pd.DataFrame, rules: list, default_val: float=1.0) -> pd.Series:
        """우선순위대로 date_mult를 채움"""
        out = pd.Series(default_val, index=g.index, dtype=float)
        assigned = pd.Series(False, index=g.index)
        for r in rules:
            name = r['name']
            mult = r['mult']
            mask = build_mask(name, g, r) & (~assigned)
            out.loc[mask] = mult
            assigned.loc[mask] = True
        return out

    def apply_month_rules(g: pd.DataFrame, rules: list, default_val: float=1.0) -> pd.Series:
        """여러 월/일 조건을 병렬로 적용(후속 규칙이 덮어씀)"""
        out = pd.Series(default_val, index=g.index, dtype=float)
        for r in rules:
            mult = r['mult']
            if 'months' in r:
                mask = g['month'].isin(r['months'])
            elif 'month' in r and ('day_from' in r or 'day_to' in r):
                # 단일 월 + 일자 범위
                mask = g['month'].eq(r['month'])
                if 'day_from' in r:
                    mask &= g['day'] >= r['day_from']
                if 'day_to' in r:
                    mask &= g['day'] <= r['day_to']
            elif 'custom_mask' in r:
                mask = r['custom_mask'](g)
            else:
                continue
            out.loc[mask] = mult
        return out

    CONFIG = {
        # 예시) 포레스트릿
        '포레스트릿': {
            'date': {
                'default': 1.0,
                'rules': [
                    {'name':'holiday',   'mult':1.35},
                    {'name':'weekend',   'mult':1.20},
                    {'name':'sandwich',  'mult':1.10},
                    {'name':'adjacent',  'mult':1.06},
                    {'name':'friday',    'mult':1.03},
                    {'name':'tuewedthu', 'mult':1.00},
                ]
            },
            'month': {
                'default': 1.0,
                'rules': [
                    {'months':[1,2,12],        'mult':1.20},
                    {'months':[4,5,6,9,10,11], 'mult':1.10},
                    {'months':[3,7,8],         'mult':0.92},
                ]
            }
        },
        '카페테리아': {
            'date': {
                'default': 1.0,
                'rules': [
                    {'name':'holiday',   'mult':1.35},
                    {'name':'weekend',   'mult':1.20},
                    {'name':'sandwich',  'mult':1.10},
                    {'name':'adjacent',  'mult':1.06},
                    {'name':'friday',    'mult':1.03},
                    {'name':'tuewedthu', 'mult':1.00},
                ]
            },
            'month': {
                'default': 1.0,
                'rules': [
                    {'months':[1,2,12],         'mult':1.20},
                    {'months':[9,10],           'mult':1.10},
                    {'months':[3,4,5,6,7,8,11], 'mult':0.92},
                ]
            }
        },
        # (예시) 미라시아 — 규칙이 다르다고 가정
        '화담숲주막': {
            'date': {
                'default': 1.0,
                'rules': [
                    {'name':'holiday',   'mult':1.35},   # 공휴일 더 강하게
                    {'name':'weekend',   'mult':1.20},
                    {'name':'sandwich',  'mult':1.10},
                    {'name':'adjacent',  'mult':1.06},
                    {'name':'tuewedthufri', 'mult':1.00},
                    {'name':'monday',    'mult':0.97},   # 월요일 약세 반영
                ]
            },
            'month': {
                'default': 1.0,
                'rules': [
                    {'months':[4,5,6,10,11],         'mult':1.20},
                    {'months':[7,8,9],           'mult':1.10},
                    {'months':[1,2,3,12], 'mult':0.92},
                ]
            }
        },
        '화담숲카페': {
            'date': {
                'default': 1.0,
                'rules': [
                    {'name':'holiday',   'mult':1.35},   # 공휴일 더 강하게
                    {'name':'weekend',   'mult':1.20},
                    {'name':'sandwich',  'mult':1.10},
                    {'name':'adjacent',  'mult':1.06},
                    {'name':'tuewedthufri', 'mult':1.00},
                    {'name':'monday',    'mult':0.97},   # 월요일 약세 반영
                ]
            },
            'month': {
                'default': 1.0,
                'rules': [
                    {'months':[4,5,6,10,11],         'mult':1.20},
                    {'months':[7,8,9],           'mult':1.10},
                    {'months':[1,2,3,12], 'mult':0.92},
                ]
            }
        },
        '미라시아': {
            'date': {
                'default': 1.0,
                'rules': [
                    {'name':'holiday',   'mult':1.35},   # 공휴일 더 강하게
                    {'name':'weekend',   'mult':1.20},
                    {'name':'sandwich',  'mult':1.10},
                    {'name':'adjacent',  'mult':1.06},
                    {'name':'friday', 'mult':1.03},
                    {'name':'montuewedthu',    'mult':1.00},   # 월요일 약세 반영
                ]
            },
            'month': {
                'default': 1.0,
                'rules': [
                    {'months':[1,2,9,10,11],         'mult':1.20},
                    {'months':[8],           'mult':1.10},
                    {'months':[4,5,6,7,12], 'mult':1.05},
                    {'months':[3], 'mult':0.92},
                ]
            }
        },
        '담하': {
            'date': {
                'default': 1.0,
                'rules': [
                    {'name':'holiday',   'mult':1.35},   # 공휴일 더 강하게
                    {'name':'weekend',   'mult':1.20},
                    {'name':'sandwich',  'mult':1.10},
                    {'name':'adjacent',  'mult':1.06},
                    {'name':'friday', 'mult':1.03},
                    {'name':'montuewedthu',    'mult':1.00},   # 월요일 약세 반영
                ]
            },
            'month': {
                'default': 1.0,
                'rules': [
                    {'months':[1,2,10,11],         'mult':1.20},
                    {'months':[4,5,6,7,8,9,12], 'mult':1.10},
                    {'months':[3], 'mult':0.92},
                ]
            }
        },
        '라그로타': {
            'date': {
                'default': 1.0,
                'rules': [
                    {'name':'holiday',   'mult':1.35},   # 공휴일 더 강하게
                    {'name':'weekend',   'mult':1.20},
                    {'name':'sandwich',  'mult':1.10},
                    {'name':'adjacent',  'mult':1.06},
                    {'name':'friday', 'mult':1.03},
                    {'name':'tuewedthu',    'mult':1.00},   # 월요일 약세 반영
                    {'name':'monday',    'mult':0.97},   # 월요일 약세 반영
                ]
            },
            'month': {
                'default': 1.0,
                'rules': [
                    {'months':[1,2,12],         'mult':1.20},
                    {'months':[4,5,6,7,8,9,10,11], 'mult':1.10},
                    {'months':[3], 'mult':0.92},
                ]
            }
        },
        '느티나무 셀프BBQ': {
            'date': {
                'default': 1.0,
                'rules': [
                    {'name':'thufri',   'mult':1.20},   # 공휴일 더 강하게
                    {'name':'montuewedsatsun',   'mult':1.00},
                ]
            },
            'month': {
                'default': 1.0,
                'rules': [
                    {'months':[4,5,6,7,9,10,11,12],         'mult':1.20},
                    {'months':[1,2,3,8], 'mult':1.00},
                ]
            }
        },
        '연회장': {
            'date': {
                'default': 1.0,
                'rules': [
                    {'name':'thufri',   'mult':1.20},   # 공휴일 더 강하게
                    {'name':'tuesat',   'mult':1.10},
                    {'name':'monsun',   'mult':1.00},
                ]
            },
            'month': {
                'default': 1.0,
                'rules': [
                    {'months':[1,2,3,4,5,6,7,8,9,10,11,12], 'mult':1.00},
                ]
            }
        },
        
    }

    
    df['date_mult']  = 1.0
    df['month_mult'] = 1.0

    for store, g in df.groupby('영업장명'):
        if store not in CONFIG:
            raise KeyError(f"CONFIG에 '{store}'가 없습니다.")
        conf = CONFIG[store]
        date_mult  = apply_date_rules(g, conf['date']['rules'],  conf['date'].get('default', 1.0))
        month_mult = apply_month_rules(g, conf['month']['rules'], conf['month'].get('default', 1.0))
        df.loc[g.index, 'date_mult']  = date_mult.values
        df.loc[g.index, 'month_mult'] = month_mult.values

    # 최종 배수 및 안정화
    df['date_weight'] = (df['date_mult'] * df['month_mult']).clip(0.7, 1.6)  # 범위는 필요 시 조정

    # 결과
    df_test['date_weight'] = df['date_weight'].values

    return df_test

## Process each test file

### Subtask:
Iterate through the list of test file paths. For each file:
    - Load the CSV into a DataFrame.
    - Apply the feature engineering function to the DataFrame.
    - Store the processed DataFrame (e.g., in a dictionary or list).


**Reasoning**:
Iterate through the test files, apply the feature engineering function to each, and store the results.



In [890]:
processed_test_dfs = {}

# Load necessary dataframes outside the loop
print(os.getcwd())  # Ensure the current working directory is set correctly
terms_df = pd.read_csv('../solar_term_2023_2025.csv')  # Load the terms CSV
holiday_df = pd.read_csv('../holidays_2023_2025.csv')
banquet_df_full = pd.read_csv('train_fin.csv') # Load the processed train data

# Prepare the banquet_df for merging (only date and type)
# banquet_df_for_merge = banquet_df_full[['영업일자', 'banquet_type']].drop_duplicates()

for file_path in test_files:
    print(f"Processing {file_path}...")
    # Load the test data
    df_test = pd.read_csv(f"../../data/test/{file_path}")

    # Apply feature engineering
    # Pass the necessary dataframes to the function
    processed_df = engineer_features(df_test, holiday_df, terms_df)

    # Store the processed DataFrame
    processed_test_dfs[file_path] = processed_df
    print(f"Finished processing {file_path}.")

# Display the first few rows of one of the processed dataframes to verify
if processed_test_dfs:
    first_file = list(processed_test_dfs.keys())[0]
    print(f"\nSample of processed {first_file}:")
    print(processed_test_dfs[first_file].columns.tolist())

/Users/garden/Desktop/lgaimers/Hackaton/DataProcessing/re_data_processed


Processing TEST_00.csv...


/var/folders/3t/x_vcdczj5fn_v1nnkppzjpwh0000gn/T/ipykernel_10003/2915347979.py:233: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('영업장명', group_keys=False).apply(mark_sandwich_adjacent)


Finished processing TEST_00.csv.
Processing TEST_01.csv...


/var/folders/3t/x_vcdczj5fn_v1nnkppzjpwh0000gn/T/ipykernel_10003/2915347979.py:233: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('영업장명', group_keys=False).apply(mark_sandwich_adjacent)


...

/var/folders/3t/x_vcdczj5fn_v1nnkppzjpwh0000gn/T/ipykernel_10003/2915347979.py:233: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('영업장명', group_keys=False).apply(mark_sandwich_adjacent)


Finished processing TEST_07.csv.
Processing TEST_08.csv...


/var/folders/3t/x_vcdczj5fn_v1nnkppzjpwh0000gn/T/ipykernel_10003/2915347979.py:233: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('영업장명', group_keys=False).apply(mark_sandwich_adjacent)


Finished processing TEST_08.csv.
Processing TEST_09.csv...
Finished processing TEST_09.csv.

Sample of processed TEST_00.csv:
['영업일자', '영업장명_메뉴명', '매출수량', '영업장명', '메뉴명', 'year', 'month', 'day', 'weekday', 'season', 'quarter', 'solar_term', 'season_sum', 'solar_term_sum', 'quarter_sum', 'is_holiday', 'is_weekend', 'menu_rank', 'is_sandwich', 'is_before_holiday', 'is_after_holiday', 'menu_category', 'avg_sales_all_days', 'avg_sales_nonzero_days', 'zero_sales_day_ratio', 'var_sales_nonzero_days', 'demand_volatility', 'menu_date_weight', 'avg_sales_nonzero_monthly', 'avg_sales_nonzero_seasonly', 'avg_sales_nonzero_weekday', 'weekday_score', 'month_score', 'date_weight']


/var/folders/3t/x_vcdczj5fn_v1nnkppzjpwh0000gn/T/ipykernel_10003/2915347979.py:233: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('영업장명', group_keys=False).apply(mark_sandwich_adjacent)


## Save processed test data (optional)

### Subtask:
Save each processed test DataFrame to a new CSV file.


**Reasoning**:
Iterate through the processed test dataframes and save each one to a CSV file with an added suffix.



In [891]:
import os

output_dir = "./test_processed_fin/" # Save in the current directory

for filename, df_processed in processed_test_dfs.items():
    # Construct output filename by adding '_processed' before the extension
    base, ext = os.path.splitext(filename)
    output_filename = f"{base}_processed{ext}"
    output_path = os.path.join(output_dir, output_filename)

    # Save the DataFrame to CSV
    df_processed.to_csv(output_path, index=False, encoding="utf-8-sig")
    

    print(f"Saved processed {filename} to {output_path}")

Saved processed TEST_00.csv to ./test_processed_fin/TEST_00_processed.csv
Saved processed TEST_01.csv to ./test_processed_fin/TEST_01_processed.csv
Saved processed TEST_02.csv to ./test_processed_fin/TEST_02_processed.csv
Saved processed TEST_03.csv to ./test_processed_fin/TEST_03_processed.csv
Saved processed TEST_04.csv to ./test_processed_fin/TEST_04_processed.csv
Saved processed TEST_05.csv to ./test_processed_fin/TEST_05_processed.csv
Saved processed TEST_06.csv to ./test_processed_fin/TEST_06_processed.csv
Saved processed TEST_07.csv to ./test_processed_fin/TEST_07_processed.csv
Saved processed TEST_08.csv to ./test_processed_fin/TEST_08_processed.csv
Saved processed TEST_09.csv to ./test_processed_fin/TEST_09_processed.csv
